# VTA 仿真

In [ ]:
def set_env(num, current_path='.'):
    import sys
    from pathlib import Path

    ROOT = Path(current_path).resolve().parents[num]
    sys.path.extend([str(ROOT/'src')]) # 设置 `tvm_book` 环境
    from tvm_book.contrib.tvm import set_env # 设置 TVM 环境

set_env(1)
from tvm_book.contrib import d2ltvm

In [ ]:
import os
import logging
import json
import numpy as np
import mxnet as mx
# from mxnet import init
# from mxnet.gluon import nn
from PIL import Image

import toml
import tvm
from tvm import autotvm, relay
import vta
from vta.top import graph_pack

input_name = 'data'
export_root = 'src/bin' # '/media/pc/data/4tb/lxw/tvm/apps/howto_deploy_cross_sim_pack' # 'src'
# lib_path = f"{export_root}/{model_name}.so"
lib_path = f"{export_root}/model.so"
tune_path = f"{export_root}/tune.log"
graph_root = 'graph_info'
model_name = "resnet18_v2"
logfile = f'outputs/{model_name}.log'
logging.basicConfig(filename=logfile,
                    # encoding='utf-8',
                    level=logging.DEBUG)

env = vta.get_env()
target = env.target
shape = (env.BATCH, 3, 224, 224)

model = mx.gluon.model_zoo.vision.get_model(model_name, pretrained=True)
relay_mod, relay_params = relay.frontend.from_mxnet(model, {input_name: shape})
with tvm.transform.PassContext(opt_level=3):
    #量化
    with relay.quantize.qconfig(global_scale=8.0, skip_conv_layers=[0], skip_dense_layer=False):
        relay_mod = relay.quantize.quantize(relay_mod, params=relay_params)
        logging.info("quantize mod: {mod['main']}")

In [ ]:
# 字典查找何时开始/结束 bit packing
pack_dict = {
    "resnet18_v1": ["nn.max_pool2d", "nn.global_avg_pool2d"],
    "resnet34_v1": ["nn.max_pool2d", "nn.global_avg_pool2d"],
    "resnet18_v2": ["nn.max_pool2d", "nn.global_avg_pool2d"],
    "resnet34_v2": ["nn.max_pool2d", "nn.global_avg_pool2d"],
    "resnet50_v2": ["nn.max_pool2d", "nn.global_avg_pool2d"],
    "resnet101_v2": ["nn.max_pool2d", "nn.global_avg_pool2d"],
}

In [ ]:
with tvm.transform.PassContext(opt_level=3):
    assert env.BLOCK_IN == env.BLOCK_OUT
    #Perform graph packing and constant folding for VTA target
    relay_prog = graph_pack(
        relay_mod["main"],
        env.BATCH,
        env.BLOCK_OUT,
        env.WGT_WIDTH,
        start_name=pack_dict[model_name][0],
        stop_name=pack_dict[model_name][1],
        # device_annot=(env.TARGET == "intelfocl"),
    )
    lib = relay.build(relay_prog, target=tvm.target.Target(target, host=env.target_host), params=relay_params)